In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
tfidf = np.load('/content/drive/MyDrive/BTT CARI-CONNECT /BTT Colabs/models/tfidf_vectorizer.pkl', allow_pickle=True)
max_scaler = np.load('/content/drive/MyDrive/BTT CARI-CONNECT /BTT Colabs/models/maxabs_scaler.pkl', allow_pickle=True)
svd = np.load('/content/drive/MyDrive/BTT CARI-CONNECT /BTT Colabs/models/svd_model.pkl', allow_pickle=True)
cmodel = np.load('/content/drive/MyDrive/BTT CARI-CONNECT /BTT Colabs/models/birch_model.pkl', allow_pickle=True)
df_match = pd.read_csv('/content/drive/MyDrive/BTT CARI-CONNECT /BTT Colabs/data/df_matching.csv', index_col = 'Unnamed: 0')
df_reduced = pd.read_csv('/content/drive/MyDrive/BTT CARI-CONNECT /BTT Colabs/data/df_reduced.csv', index_col = 'Unnamed: 0')

In [6]:
# Text Preprocessing
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import sys
!{sys.executable} -m pip install contractions
import contractions
def preprocess(s):
    s=str(s)
    s = s.lower() # lowercase
    s = contractions.fix(s) # expand contractions
    s = re.sub(r'\\n', ' ', s) # remove \n
    s = re.sub(r'http\S+', '', s) # remove url
    s = re.sub(r'<.*?>', '', s) # remove html
    s = re.sub(r'\d+', '', s) # remove numbers
    s = re.sub(r'[^\w\s]', ' ', s) # remove punctuation and special characters
    s = word_tokenize(s) # tokenize
    s = [w for w in s if w not in set(stopwords.words('english'))] # stop words
    s = [stemmer.stem(w) for w in s] # stemming
    return " ".join(s) # white spaces
def preprocess_transform(s):
  preprocessed = preprocess(s)
  transformed = tfidf.transform([preprocessed]).toarray()
  return transformed

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
!pip install langdetect
!pip install deep_translator
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from deep_translator import GoogleTranslator

DetectorFactory.seed = 0

# Input language determinate
class LanguageDetector:
    def __init__(self, text=""):
        self.text = text

    def set_text(self, text):
        self.text = text

    def detect_language(self):
        try:
            language_code = detect(self.text)
            return language_code
        except LangDetectException:
            language_code = "en"
            return language_code
def translate_to_english(text):
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        return translated
    except:
        return text
detector = LanguageDetector()

In [8]:
# column weights
title_weight=.05
subject_weight=.8
synopsis_weight=.15

In [9]:
# Clean and vectorize input
title_input = "Anansi"
subject_input = "Akan Folklore"
synopsis_input = "Anansi or Ananse (/əˈnɑːnsi/ ə-NAHN-see; literally translates to spider) is an Akan folktale character associated with stories, wisdom, knowledge, and trickery, most commonly depicted as a spider, in Akan folklore.[1] Taking the role of a trickster, he is also one of the most important characters of West African, African American and West Indian folklore. Originating in Ghana, these spider tales were transmitted to the Caribbean by way of the transatlantic slave trade.[2]  Anansi is best known for his ability to outsmart and triumph over more powerful opponents through his use of cunning, creativity and wit.[3] Despite taking on a trickster role, Anansi often takes centre stage in stories and is commonly portrayed as both the protagonist and antagonist."
detector.set_text(synopsis_input)
detect_input = detector.detect_language()
synopsis_input = translate_to_english(synopsis_input)
lang_input = detect_input # detect language, or selected language

In [10]:
clean_title_input = title_weight*preprocess_transform(title_input)
clean_subject_input = subject_weight*preprocess_transform(subject_input)
clean_synopsis_input = synopsis_weight*preprocess_transform(synopsis_input)
input_df = pd.DataFrame(np.concatenate([clean_title_input, clean_subject_input, clean_synopsis_input], axis=1))

In [11]:
# scale input
input_df = max_scaler.transform(input_df)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(


In [12]:
# reduce dimensions of input
svd_input = svd.transform(input_df)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but TruncatedSVD was fitted with feature names
  warnings.warn(


In [13]:
red_input = svd_input

In [14]:
# predict test input
input_pred = cmodel.predict(red_input)[0]

In [15]:
# vector subset where label matches
subset = df_match[df_match['label'] == input_pred]

In [16]:
# get subset
sim_df = df_reduced.copy()
sim_subset = sim_df.loc[subset.index]

In [17]:
# calculate cosine similarity within that cluster
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
simscore = cosine_similarity
similarity = simscore(red_input, sim_subset).T #input
similarity

array([[0.45674392],
       [0.46822016],
       [0.67631379],
       [0.52412832],
       [0.50367231],
       [0.31333569],
       [0.22510147],
       [0.31211111],
       [0.53825146],
       [0.44198038],
       [0.69648897],
       [0.0907768 ],
       [0.19630366],
       [0.52673414],
       [0.54851469],
       [0.47810201],
       [0.47215413],
       [0.14705573],
       [0.2514256 ],
       [0.18340826],
       [0.64308556],
       [0.24989326],
       [0.34956625],
       [0.465373  ],
       [0.59439651],
       [0.39483388],
       [0.5343515 ],
       [0.40367092],
       [0.10222779],
       [0.1178229 ],
       [0.13676142],
       [0.21456368],
       [0.34082982],
       [0.5993023 ],
       [0.6718296 ],
       [0.68187432],
       [0.56187598],
       [0.56138063],
       [0.27744764],
       [0.39850816],
       [0.12332258],
       [0.20478356],
       [0.38998698],
       [0.52297865],
       [0.08270452],
       [0.13157323],
       [0.85579493],
       [0.624

In [18]:
# append similarity to dataframe
subset['similarity'] = similarity

<ipython-input-18-bbfe0a642cef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['similarity'] = similarity


In [19]:
subset.loc[df_match['language'] == lang_input, 'similarity']+=1 # prioritize same language

In [20]:
# top 5 similar
top_subset = subset.sort_values(by='similarity', ascending=False).head(5)
top_subset

,title,subjects,publisher,synopsis,language,label,similarity
1183,A String of Pearls: Caribbean Travels and Beyond,NaN,Olympia Publishers,Product Description<br/><br/><br/>Meet the Car...,en,22,1.867137
1280,Adult education in the Caribbean at the turn o...,NaN,UNESCO,<p>Burnout is common among doctors in the West...,en,22,1.858134
879,Caribbean Flame,Romance,Constable Robinson,"Damon Alexander cannot resist Ramona King, alt...",en,22,1.855795
1449,The Caribbean Raj,NaN,"Ian Randle Publishers,Jamaica",This book is a collection of poetry. It brings...,en,22,1.852264
3730,Caribbean Dreams,NaN,UNKNO,"Review "". . . a vastly entertaining work ...",en,22,1.839313


In [21]:
top_index = top_subset.index
top_index

Index([1183, 1280, 879, 1449, 3730], dtype='int64')

In [22]:
# get information of top 5
top_titles = []
top_subjects = []
top_synopsis = []
top_publishers = []
top_languages = []

In [23]:
def book_info(index):
  title = df_match.loc[index]['title']
  subjects = df_match.loc[index]['subjects']
  synopsis = df_match.loc[index]['synopsis']
  publisher = df_match.loc[index]['publisher']
  language = df_match.loc[index]['language']
  return title, subjects, synopsis, publisher, language

In [24]:
for i in range(len(top_index)):
  top_titles.append(book_info(top_index[i])[0])
  top_subjects.append(book_info(top_index[i])[1])
  top_synopsis.append(book_info(top_index[i])[2])
  top_publishers.append(book_info(top_index[i])[3])
  top_languages.append(book_info(top_index[i])[4])
  print(book_info(top_index[i])[0])

A String of Pearls: Caribbean Travels and Beyond
Adult education in the Caribbean at the turn of the century
Caribbean Flame
The Caribbean Raj
Caribbean Dreams


In [25]:
# GROQ API to prompt LLAMA
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.0 MB/s eta 0:00:00


In [26]:
%env GROQ_API_KEY=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

env: GROQ_API_KEY=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


In [27]:
from groq import Groq

In [28]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [29]:
chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": f"You are a knowledgable and straightforward assistant with experience in literature and literary analysis, and knowledge of the publishing industry. You are explaining to authors which books are most similar to theirs using criteria such as themes, plot, character, setting, and tone. Your responses are concise and academic, strictly providing lists and explanations. Structure the response in a list, where each entry is formatted, with elaboration as necessary. (Do not mention the books' alphabetical labels A B C D E) : \n (Rank number). (Publisher) \n Published in (Language)\n Explanation: (Publisher) published (Title), a (Subject) book that is similar to your book because (Explanation, 3 sentence justification of the book similarities). Therefore, your book appeals to (Publisher)'s sector of the market."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": f"Using the input book information as a reference, compare and rank the similarity of books A, B, C, D, and E to the reference. Use literary analysis to evaluate each book based on thematic alignment, such as central conflict, character focus, and setting, and stylistic features such as narrative tone and diction. \n Provide a list of the respective publishers of the books in descending order of book similarity to the reference book with a 3 sentence justification of the book similarities and ranking, citing specific aspects of the book information and synopsis that supports your assessment. Structure the response in a list, where each entry is formatted, with elaboration as necessary. (Do not mention the books' alphabetical labels A B C D E) : \n (Rank number). (Publisher) \n Published in (Language)\n Explanation: (Publisher) published (Title), a (Subject) book that is similar to your book because (Explanation, 3 sentence justification of the book similarities). Therefore, your book appeals to (Publisher)'s sector of the market. \n \n Book Reference: \n Title: {title_input}. \n Subject: {subject_input} \n Synopsis: {synopsis_input} \n \n Book A: \n Title: {top_titles[0]} \n Subject: {top_subjects[0]} \n Synopsis: {top_synopsis[0]} \n Publisher: {top_publishers[0]} \n Language: {top_languages[0]} \n \n Book B: \n Title: {top_titles[1]} \n Subject: {top_subjects[1]} \n Synopsis: {top_synopsis[1]} \n Publisher: {top_publishers[1]} \n Language: {top_languages[1]} \n \n Book C: \n Title: {top_titles[2]} \n Subject: {top_subjects[2]} \n Synopsis: {top_synopsis[2]} \n Publisher: {top_publishers[2]} \n Language: {top_languages[2]} \n \n Book D: \n Title: {top_titles[3]} \n Subject: {top_subjects[3]} \n Synopsis: {top_synopsis[3]} \n Publisher: {top_publishers[3]} \n Language: {top_languages[3]} \n \n Book E: \n Title: {top_titles[4]} \n Subject: {top_subjects[4]} \n Synopsis: {top_synopsis[4]} \n Publisher: {top_publishers[4]} \n Language: {top_languages[4]}"

        }

    ],

    # The language model which will generate the completion.
    model= "llama3-70b-8192",#"llama3-70b-8192", "llama3-8b-8192", "llama-3.1-70b-versatile", "llama-3.1-8b-instant"

    #
    # Optional parameters
    #

    # Controls randomness: lowering results in less random completions.
    # As the temperature approaches zero, the model will become deterministic
    # and repetitive.
    temperature=0.3,

    # The maximum number of tokens to generate. Requests can use up to
    # 32,768 tokens shared between prompt and completion.
    max_tokens=8000,

    # Controls diversity via nucleus sampling: 0.5 means half of all
    # likelihood-weighted options are considered.
    top_p=1,

    # A stop sequence is a predefined or user-specified text string that
    # signals an AI to stop generating content, ensuring its responses
    # remain focused and concise. Examples include punctuation marks and
    # markers like "[end]".
    stop=None,

    # If set, partial message deltas will be sent.
    stream=False,
)

In [30]:
# Print the completion returned by the LLM.
output = chat_completion.choices[0].message.content

In [31]:
output = GoogleTranslator(source='auto', target=lang_input).translate(output)

In [32]:
print(output)

Here is the list of publishers in descending order of book similarity to the reference book "Anansi":

1. Ian Randle Publishers, Jamaica
Published in: en
Explanation: Ian Randle Publishers published "The Caribbean Raj", a poetry collection book that is similar to "Anansi" because it explores the cultural heritage and identity of the Caribbean region, much like the Akan folklore character Anansi. The themes of colonialism, cultural identity, and the blending of cultures are common to both books. Therefore, your book appeals to Ian Randle Publishers' sector of the market.

2. Olympia Publishers
Published in: en
Explanation: Olympia Publishers published "A String of Pearls: Caribbean Travels and Beyond", a travelogue book that is similar to "Anansi" because it explores the cultural diversity and richness of the Caribbean region. The book's focus on the author's personal experiences and encounters with local cultures mirrors Anansi's trickster role in navigating different cultural contexts